# Experiment Introduction

Background, 
In order to do ablation experiment, we have diffculty like
1. most of them are same, we have to repeat many times.
2. too many model, config, code hard to manage and save. too messy


For all kind of SSL training workflow, we have to define the hyperparameters includes 4 aspect,
Dataset
Model
Training
Saving COnfig



How to Use this Experiment?



# Library

In [1]:
# Import config
import sys 
sys.path.append("C:/Users/isxzl/OneDrive/Code/AutoSSL")
import yaml
from torchvision.transforms import RandomRotation,GaussianBlur,ColorJitter
from autoSSL.evaluate import eval_KNN,eval_linear,eval_KNNplot,pipe_collate
from autoSSL.models import BarlowTwins, BYOL, MoCo, SimCLR, SimSiam, VICReg ,Toymodel, pipe_model 
from autoSSL.utils import embedding_feature,ck_callback,dict2transformer,trans2multi,join_dir,ContinuousCSVLogger  
from autoSSL.data import PipeDataset
from autoSSL.train import Trainer

from lightly.transforms import SimCLRTransform
from lightly.transforms.rotation import random_rotation_transform
import torchvision.transforms as T
import pytorch_lightning as pl
pl.seed_everything(0)

C:\Users\isxzl\anaconda3\envs\AutoGPT\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Global seed set to 0


0

# Import Global Baseline Config

In [2]:
# Load the YAML file
with open('global.yaml', 'r') as file:
    global_config = yaml.safe_load(file)

# Write your experiment notebook name here
global_config["experiment"]="views VS model"   
global_config["dataset_dir"]="../Datasets/cifar10/train/"
global_config["prjhead_dim"]=[[512,2048, "BN","RELU"],[2048,2048,None,None]]    

global_config["loss_func"]="BarlowTwinsLoss"     
global_config["view_model"]="None"     
global_config["view"]=2 
global_config["stop_gradient"]=False   
global_config["optimizer"]="SGD"      
global_config["schedule"]="cos"   
global_config["model"]="Toymodel"
global_config["batch_size"]=128
global_config["input_size"]=32
global_config["max_epochs"]=50

#global_config["backbone"]="resnet18_pretrained"

# Define global Training Augmentation
global_SSL_augmentation=T.Compose([
    T.RandomResizedCrop(size=global_config["input_size"], scale=(0.08, 1.0)),
    T.RandomApply([T.RandomRotation(degrees=90)], p=0),
    T.RandomHorizontalFlip(p= 0.5),
    T.RandomVerticalFlip(p=0),
    T.RandomApply([T.ColorJitter(
            brightness=0.4,contrast=0.4,saturation=0.4, hue=0.1)], p=0.8),
    T.RandomGrayscale(p=0.2),
    T.ToTensor(),
    T.Normalize(mean= [0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


# Define the Testing Augmentation
test_SSL_augmentation=T.Compose([
    T.ToTensor(),
    T.Normalize(mean= [0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

p_knndata= PipeDataset(input_dir=global_config["path_to_test_cifar10"], 
        augmentation=trans2multi(test_SSL_augmentation,view=1), 
        batch_size=global_config["batch_size"],num_workers=global_config["num_workers"]).dataloader
p_knndata=[p_knndata,10]  # The second number is the classes number of this datasets


## Config 1 Baseline for everything (Negative)

In [ ]:
pd=[[2048,512, "BN","RELU"],[512,2048,None,None]]    
st=True


for baseline in [[pd,st,True],[[],False,False],[pd,False,True],[[],st,False]]:
    # MAKE YOUR OWN CONFIG
    config=global_config.copy()
    # Fill the config
    SSL_augmentation=global_SSL_augmentation
    config["view"]=2
    config["view_model"]= "None"
    config["name"]=f"LNeg_VMNone_PR{baseline[2]}_st{baseline[1]}_b128_m50_v{4}"
    config["loss_func"]="NegativeCosineSimilarity" 
    config["prjhead_dim"]=[[512,2048, "BN","RELU"],[2048,2048,"BN",None]]    
    config["stop_gradient"]= baseline[1]
    config["predhead_dim"]= baseline[0]
    config["max_epochs"]= 50
    # THIS IS THE CODE TO LOAD DATASET
    pdata= PipeDataset(config=config,augmentation=trans2multi(SSL_augmentation,view=config["view"]))
    
    # THIS IS THE CODE TO LOAD MODEL
    #pmodel=pipe_model(config=config) 
    pmodel=pipe_model(config=config,MonitoringbyKNN=p_knndata) # All save the validation
    
 
    # Use this if you want to START a train
    trainer=Trainer(config, model_mode="start",check_val_every_n_epoch =5) #precision='16-mixed',
    trainer.fit(pmodel, pdata.dataloader,)  
    
    # Use this if you want to CONTINUE a train
    #trainer=Trainer(config, model_mode="continue", extra_epoch=1,precision='16-mixed')
    #trainer.fit(pmodel, pdata.dataloader,ckpt_path="latest")  
    print(pmodel.debug)
    del pdata
    del pmodel
    del trainer    

## View 4 SOTA View

In [ ]:

for baseline0 in ["mean_n" ,"fastsim", "pair-pair" , "1_n" ,"1_fastsim"]:
    
    
    pd=[[2048,512, "BN","RELU"],[512,2048,None,None]]    
    st=True
    pd2=[[2048,512, "BN","RELU"],[512,2048,"BN",None]]  
    for baseline in [[pd,st,"TruewithBN"]]: #,[[],False,False],[pd,False,"TruewithBN"],[[],st,False]
        # MAKE YOUR OWN CONFIG
        config=global_config.copy()
        # Fill the config
        SSL_augmentation=global_SSL_augmentation
        config["view"]=4
        config["view_model"]= baseline0
        config["name"]=f"LNeg_VM{baseline0}_PR{baseline[2]}_st{baseline[1]}_b128_m50_v{4}"
        config["loss_func"]="NegativeCosineSimilarity" 
        config["prjhead_dim"]=[[512,2048, "BN","RELU"],[2048,2048,"BN",None]]    
        config["stop_gradient"]= baseline[1]
        config["predhead_dim"]= baseline[0]
        config["max_epochs"]= 50
        # THIS IS THE CODE TO LOAD DATASET
        pdata= PipeDataset(config=config,augmentation=trans2multi(SSL_augmentation,view=config["view"]))

        # THIS IS THE CODE TO LOAD MODEL
        #pmodel=pipe_model(config=config) 
        pmodel=pipe_model(config=config,MonitoringbyKNN=p_knndata) # All save the validation


        # Use this if you want to START a train
        trainer=Trainer(config, model_mode="start",check_val_every_n_epoch =5) #precision='16-mixed',
        trainer.fit(pmodel, pdata.dataloader,)  

        # Use this if you want to CONTINUE a train
        #trainer=Trainer(config, model_mode="continue", extra_epoch=1,precision='16-mixed')
        #trainer.fit(pmodel, pdata.dataloader,ckpt_path="latest")  
        print(pmodel.debug)
        del pdata
        del pmodel
        del trainer    

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
C:\Users\isxzl\anaconda3\envs\AutoGPT\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:615: UserWarning: Checkpoint directory C:\Users\isxzl\OneDrive\Code\AutoSSL\experiment\experiment_checkpoints\views VS model\LNeg_VMmean_n_PRTruewithBN_stTrue_b128_m50_v4 exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type                     | Params
--------

Epoch 4: 100%|██████████████████████████████████████████████████████████████| 390/390 [00:42<00:00,  9.24it/s, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 9: 100%|██████████████████████████████████████████| 390/390 [00:43<00:00,  8.90it/s, v_num=0, kNN_accuracy=41.20]
Validation: 0it [00:00, ?it/s]
Epoch 14: 100%|█████████████████████████████████████████| 390/390 [00:38<00:00, 10.13it/s, v_num=0, kNN_accuracy=49.60]
Validation: 0it [00:00, ?it/s]
Epoch 19: 100%|█████████████████████████████████████████| 390/390 [00:41<00:00,  9.45it/s, v_num=0, kNN_accuracy=53.30]
Validation: 0it [00:00, ?it/s]
Epoch 24: 100%|█████████████████████████████████████████| 390/390 [00:45<00:00,  8.58it/s, v_num=0, kNN_accuracy=57.10]
Validation: 0it [00:00, ?it/s]
Epoch 29: 100%|█████████████████████████████████████████| 390/390 [00:40<00:00,  9.64it/s, v_num=0, kNN_accuracy=62.00]
Validation: 0it [00:00, ?it/s]
Epoch 34: 100%|█████████████████████████████████████████| 390/390 [00:38<00:00, 10.01it/s, v_n

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 49: 100%|█████████████████████████████████████████| 390/390 [00:51<00:00,  7.56it/s, v_num=0, kNN_accuracy=71.30]
{'stop': 8, 'optim': [50000, 128, 50]}


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
C:\Users\isxzl\anaconda3\envs\AutoGPT\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:615: UserWarning: Checkpoint directory C:\Users\isxzl\OneDrive\Code\AutoSSL\experiment\experiment_checkpoints\views VS model\LNeg_VMfastsim_PRTruewithBN_stTrue_b128_m50_v4 exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type                     | Params
-------

Epoch 4: 100%|██████████████████████████████████████████████████████████████| 390/390 [00:41<00:00,  9.42it/s, v_num=1]
Validation: 0it [00:00, ?it/s]
Epoch 9: 100%|██████████████████████████████████████████| 390/390 [00:40<00:00,  9.56it/s, v_num=1, kNN_accuracy=37.70]
Validation: 0it [00:00, ?it/s]
Epoch 14: 100%|█████████████████████████████████████████| 390/390 [00:40<00:00,  9.75it/s, v_num=1, kNN_accuracy=46.00]
Validation: 0it [00:00, ?it/s]
Epoch 19: 100%|█████████████████████████████████████████| 390/390 [00:40<00:00,  9.61it/s, v_num=1, kNN_accuracy=50.90]
Validation: 0it [00:00, ?it/s]
Epoch 24: 100%|█████████████████████████████████████████| 390/390 [00:41<00:00,  9.33it/s, v_num=1, kNN_accuracy=54.30]
Validation: 0it [00:00, ?it/s]
Epoch 29: 100%|█████████████████████████████████████████| 390/390 [00:42<00:00,  9.25it/s, v_num=1, kNN_accuracy=57.00]
Validation: 0it [00:00, ?it/s]
Epoch 34: 100%|█████████████████████████████████████████| 390/390 [00:45<00:00,  8.53it/s, v_n

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 49: 100%|█████████████████████████████████████████| 390/390 [01:02<00:00,  6.19it/s, v_num=1, kNN_accuracy=67.60]
{'stop': 4, 'optim': [50000, 128, 50]}


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
C:\Users\isxzl\anaconda3\envs\AutoGPT\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:615: UserWarning: Checkpoint directory C:\Users\isxzl\OneDrive\Code\AutoSSL\experiment\experiment_checkpoints\views VS model\LNeg_VMpair-pair_PRTruewithBN_stTrue_b128_m50_v4 exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type                     | Params
-----

Epoch 4: 100%|██████████████████████████████████████████████████████████████| 390/390 [00:44<00:00,  8.84it/s, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 9: 100%|██████████████████████████████████████████| 390/390 [00:45<00:00,  8.62it/s, v_num=0, kNN_accuracy=38.50]
Validation: 0it [00:00, ?it/s]
Epoch 14: 100%|█████████████████████████████████████████| 390/390 [00:41<00:00,  9.31it/s, v_num=0, kNN_accuracy=43.00]
Validation: 0it [00:00, ?it/s]
Epoch 19: 100%|█████████████████████████████████████████| 390/390 [00:49<00:00,  7.92it/s, v_num=0, kNN_accuracy=48.20]
Validation: 0it [00:00, ?it/s]
Epoch 22:   0%|                                                   | 0/390 [00:00<?, ?it/s, v_num=0, kNN_accuracy=52.20]

In [ ]:

for baseline in [2]:
    # MAKE YOUR OWN CONFIG
    config=global_config.copy()
    # Fill the config
    SSL_augmentation=global_SSL_augmentation
    config["name"]=f"fastsim_BT_b128_m50_v{str(baseline)}"
    config["view"]=2
    config["view_model"]= "None" 
    # THIS IS THE CODE TO LOAD DATASET
    pdata= PipeDataset(config=config,augmentation=trans2multi(SSL_augmentation,view=config["view"]))
    
    # THIS IS THE CODE TO LOAD MODEL
    #pmodel=pipe_model(config=config) 
    pmodel=pipe_model(config=config,MonitoringbyKNN=p_knndata) # All save the validation
    
 
    # Use this if you want to START a train
    trainer=Trainer(config, model_mode="start",check_val_every_n_epoch =5) #precision='16-mixed',
    trainer.fit(pmodel, pdata.dataloader,)  
    
    # Use this if you want to CONTINUE a train
    #trainer=Trainer(config, model_mode="continue", extra_epoch=1,precision='16-mixed')
    #trainer.fit(pmodel, pdata.dataloader,ckpt_path="latest")  
    print(pmodel.debug)
    del pdata
    del pmodel
    del trainer    

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
C:\Users\isxzl\anaconda3\envs\AutoGPT\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:615: UserWarning: Checkpoint directory C:\Users\isxzl\OneDrive\Code\AutoSSL\experiment\experiment_checkpoints\views VS model\fastsim_BT_b128_m50_v2 exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type            | Params
----------------------------------------

Epoch 4: 100%|██████████████████████████████████████████████████████████████| 390/390 [00:52<00:00,  7.41it/s, v_num=5]
Validation: 0it [00:00, ?it/s]
Epoch 5:  11%|████▊                                      | 44/390 [00:11<01:33,  3.70it/s, v_num=5, kNN_accuracy=34.20]

## Config1: Baseline of Cifar with Batch 512

In [ ]:

for baseline in [4,12]:
    # MAKE YOUR OWN CONFIG
    config=global_config.copy()
    # Fill the config
    SSL_augmentation=global_SSL_augmentation
    config["name"]=f"fastsim_BT_Baseline_b128_m50_v{str(baseline)}"

    config["view"]=baseline
    config["view_model"]= "fastsim" 
    # THIS IS THE CODE TO LOAD DATASET
    pdata= PipeDataset(config=config,augmentation=dict2transformer(SSL_augmentation,view=config["view"]))
    
    # THIS IS THE CODE TO LOAD MODEL
    #pmodel=pipe_model(config=config) 
    pmodel=pipe_model(config=config,MonitoringbyKNN=p_knndata) # All save the validation
    
 
    # Use this if you want to START a train
    trainer=Trainer(config, model_mode="start",check_val_every_n_epoch =5) #precision='16-mixed',
    trainer.fit(pmodel, pdata.dataloader,)  
    
    # Use this if you want to CONTINUE a train
    #trainer=Trainer(config, model_mode="continue", extra_epoch=1,precision='16-mixed')
    #trainer.fit(pmodel, pdata.dataloader,ckpt_path="latest")  
    print(pmodel.debug)
    del pdata
    del pmodel
    del trainer    

## Config2: Use SGD + Cos

In [ ]:

for baseline in [4,12]:
    # MAKE YOUR OWN CONFIG
    config=global_config.copy()
    # Fill the config
    SSL_augmentation=global_SSL_augmentation.copy()
    config["name"]=f"pair-pair_BT_Baseline_b128_m50_v{str(baseline)}"

    config["view"]=baseline
    config["view_model"]= "pair-pair" 
    # THIS IS THE CODE TO LOAD DATASET
    pdata= PipeDataset(config=config,augmentation=dict2transformer(SSL_augmentation,view=config["view"]))
    
    # THIS IS THE CODE TO LOAD MODEL
    #pmodel=pipe_model(config=config) 
    pmodel=pipe_model(config=config,MonitoringbyKNN=p_knndata) # All save the validation
    
 
    # Use this if you want to START a train
    trainer=Trainer(config, model_mode="start",check_val_every_n_epoch =5) #precision='16-mixed',
    trainer.fit(pmodel, pdata.dataloader,)  
    
    # Use this if you want to CONTINUE a train
    #trainer=Trainer(config, model_mode="continue", extra_epoch=1,precision='16-mixed')
    #trainer.fit(pmodel, pdata.dataloader,ckpt_path="latest")  
    print(pmodel.debug)
    del pdata
    del pmodel
    del trainer    

## Check Perfect

In [ ]:

for baseline in [4,12]:
    # MAKE YOUR OWN CONFIG
    config=global_config.copy()
    # Fill the config
    SSL_augmentation=global_SSL_augmentation
    config["name"]=f"1_n_BT_Baseline_b128_m50_v{str(baseline)}"

    config["view"]=baseline
    config["view_model"]= "1_n"   # 1_n # mean_n #1_fastsim
    # THIS IS THE CODE TO LOAD DATASET
    pdata= PipeDataset(config=config,augmentation=dict2transformer(SSL_augmentation,view=config["view"]))
    
    # THIS IS THE CODE TO LOAD MODEL
    #pmodel=pipe_model(config=config) 
    pmodel=pipe_model(config=config,MonitoringbyKNN=p_knndata) # All save the validation
    
 
    # Use this if you want to START a train
    trainer=Trainer(config, model_mode="start",check_val_every_n_epoch =5) #precision='16-mixed',
    trainer.fit(pmodel, pdata.dataloader,)  
    
    # Use this if you want to CONTINUE a train
    #trainer=Trainer(config, model_mode="continue", extra_epoch=1,precision='16-mixed')
    #trainer.fit(pmodel, pdata.dataloader,ckpt_path="latest")  
    print(pmodel.debug)
    del pdata
    del pmodel
    del trainer    


## Improved Trainer

In [ ]:

for baseline in [4,12]:
    # MAKE YOUR OWN CONFIG
    config=global_config.copy()
    # Fill the config
    SSL_augmentation=global_SSL_augmentation
    config["name"]=f"mean_n_BT_Baseline_b128_m50_v{str(baseline)}"

    config["view"]=baseline
    config["view_model"]= "mean_n"   # 1_n # mean_n #1_fastsim
    # THIS IS THE CODE TO LOAD DATASET
    pdata= PipeDataset(config=config,augmentation=dict2transformer(SSL_augmentation,view=config["view"]))
    
    # THIS IS THE CODE TO LOAD MODEL
    #pmodel=pipe_model(config=config) 
    pmodel=pipe_model(config=config,MonitoringbyKNN=p_knndata) # All save the validation
    
 
    # Use this if you want to START a train
    trainer=Trainer(config, model_mode="start",check_val_every_n_epoch =5) #precision='16-mixed',
    trainer.fit(pmodel, pdata.dataloader,)  
    
    # Use this if you want to CONTINUE a train
    #trainer=Trainer(config, model_mode="continue", extra_epoch=1,precision='16-mixed')
    #trainer.fit(pmodel, pdata.dataloader,ckpt_path="latest")  
    print(pmodel.debug)
    del pdata
    del pmodel
    del trainer    


## Config 5

In [ ]:
 
for baseline in [4,12]:
    # MAKE YOUR OWN CONFIG
    config=global_config.copy()
    # Fill the config
    SSL_augmentation=global_SSL_augmentation
    config["name"]=f"1_fastsim_BT_Baseline_b128_m50_v{str(baseline)}"

    config["view"]=baseline
    config["view_model"]= "1_fastsim"   # 1_n # mean_n #
    # THIS IS THE CODE TO LOAD DATASET
    pdata= PipeDataset(config=config,augmentation=dict2transformer(SSL_augmentation,view=config["view"]))
    
    # THIS IS THE CODE TO LOAD MODEL
    #pmodel=pipe_model(config=config) 
    pmodel=pipe_model(config=config,MonitoringbyKNN=p_knndata) # All save the validation
    
 
    # Use this if you want to START a train
    trainer=Trainer(config, model_mode="start",check_val_every_n_epoch =5) #precision='16-mixed',
    trainer.fit(pmodel, pdata.dataloader,)  
    
    # Use this if you want to CONTINUE a train
    #trainer=Trainer(config, model_mode="continue", extra_epoch=1,precision='16-mixed')
    #trainer.fit(pmodel, pdata.dataloader,ckpt_path="latest")  
    print(pmodel.debug)
    del pdata
    del pmodel
    del trainer    


# Evaluation

In [ ]:

test_augmentation = {
    'RandomResizedCrop': {'size': (global_config["input_size"], global_config["input_size"])},
    'ToTensor': {},
    'Normalize': {"mean": [0.485, 0.456, 0.406], "std": [0.229, 0.224, 0.225]}
}
collate =pipe_collate(address="experiment_checkpoints/paper_barlowtwins/", reg="BT_Baseline_512_[0-9]+")

pdata = PipeDataset(input_dir=global_config["path_to_test_cifar10"],
    augmentation=dict2transformer(test_augmentation,view=1))


In [ ]:
aaa=eval_linear(pdata, models=collate, device=global_config["device"])


In [ ]:
aaa